In [35]:

import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np

In [36]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [37]:
url = 'https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-'
browser.visit(url)

In [38]:
html = browser.html
html_soup = soup(html, 'html.parser')

In [39]:
extracted_titles = html_soup.find_all('div','i9-e-table__container i9-e-table__container--light')

In [40]:
out = []

for i in extracted_titles[0].find_all('tr'):
    j = i.find_all('td')
    out.append([k.get_text().strip() for k in j])

In [41]:
out = out[1:]

In [42]:
df=pd.DataFrame(out,columns=["Year","Annual Average CPI(-U)","rate_of_inflation"])
df

,Year,Annual Average CPI(-U),rate_of_inflation
0,1913,9.9,
1,1914,10.0,1.3%
2,1915,10.1,0.9%
3,1916,10.9,7.7%
4,1917,12.8,17.8%
...,...,...,...
105,2018,251.1,2.4%
106,2019,255.7,1.8%
107,2020,258.8,1.2%
108,2021,271.0,4.7%


In [43]:
df=df.replace("2022*","2022")
df["Year"]=df["Year"].astype(int)
df["Annual Average CPI(-U)"]=df["Annual Average CPI(-U)"].astype(float)

In [44]:
df.dtypes

Year                        int32
Annual Average CPI(-U)    float64
rate_of_inflation          object
dtype: object

In [45]:
df["rate_of_inflation"] = df["rate_of_inflation"].str.replace("%", "")
df["rate_of_inflation"] = df["rate_of_inflation"].replace('', np.nan)
df['rate_of_inflation'] = df['rate_of_inflation'].astype(float)






In [46]:
df.dtypes

Year                        int32
Annual Average CPI(-U)    float64
rate_of_inflation         float64
dtype: object

In [47]:
df=round(df.groupby("Year").mean().astype(float),3)

In [48]:
df.sort_values("Year",ascending=False)

,Annual Average CPI(-U),rate_of_inflation
Year,,
2022,294.4,8.6
2021,271.0,4.7
2020,258.8,1.2
2019,255.7,1.8
2018,251.1,2.4
...,...,...
1917,12.8,17.8
1916,10.9,7.7
1915,10.1,0.9


In [55]:
df.loc[df["rate_of_inflation"]>0]

,Annual Average CPI(-U),rate_of_inflation
Year,,
1914,10.0,1.3
1915,10.1,0.9
1916,10.9,7.7
1917,12.8,17.8
1918,15.0,17.3
...,...,...
2018,251.1,2.4
2019,255.7,1.8
2020,258.8,1.2


In [57]:
df=df.dropna()


In [60]:
df = df.rename(columns={'Year': 'Year', 'Annual Average CPI(-U)': 'yearly_increase', 'rate_of_inflation': 'yearly_inflation'})


In [61]:
df.sort_values("Year",ascending=False)

,yearly_increase,yearly_inflation
Year,,
2022,294.4,8.6
2021,271.0,4.7
2020,258.8,1.2
2019,255.7,1.8
2018,251.1,2.4
...,...,...
1918,15.0,17.3
1917,12.8,17.8
1916,10.9,7.7


In [62]:
df.to_csv("Inflation_CPI_All.csv")